Fukushima Tabulate Example using SQLite Database
==========================================
This example illustrates basic queries used to list all of the data in the SQLite database in HTML tables.  The converter created the following four types of records:

* **Source**: Paths to source and data dictionary files
* **Experiments**: One record per flight
* **Observations**: Each observation taken during each of the three flights
* **Units**: One record for each measurement to provide efficient representation of units and a description

All records will be output in a single table for each record type before all relationships are output in the final table.

Make sure you have a Sina kernel selected before you begin.

You can reconfigure this example to use your copy of the database by modifying **DB_PATH** in the next cell to point to the root directory of your copy of the Fukushima database.  Once the path is set properly, run the next cell.

In [ ]:
import os

# Root directory containing the fukushima.sqlite database
DB_PATH = '/collab/usr/gapps/wf/examples/data/fukushima/fukushima.sqlite'

print('Path to the database is configured.  Ready to run the next cell.')

Prepare to Perform Queries and Display Results
======================================
Create the data access object factory in preparation for record and relationship queries.  Also import the relevant Sina package(s) and define a function for displaying the data in an HTML table.

In [ ]:
import IPython.display as ipyd
import sina.datastores.sql as sina_sql
import tabulate

# Create the data access object factory.
factory = sina_sql.DAOFactory(DB_PATH)

def display_table(rows):
    """Displays a table assuming the first row contains column headings"""
    ipyd.display(ipyd.HTML(tabulate.tabulate(rows, tablefmt='html')))

print('Ready to run the next cell.')

Load and Display the Source Record
=============================
Load and display records of type "source".  There is actually only one record.

In [ ]:
print('Loading source record(s).')

# Extract source record(s) from the database
source_recs = factory.createRecordDAO().get_all_of_type("source")

# Build a list of table entries, one source per table row
rows = [['Source Id', 'File URI', 'File Type', 'File Tag']]

for source in source_recs:
    id_ = source['id']

    for file_ in source['files']:
        uri = file_['uri']
        file_type = file_['mimetype']
        file_tags = file_['tags'][0]  # There's only one tag per file
        rows.append([id_, uri, file_type, file_tags])

# Display the rows in an HTML table
display_table(rows)

Load and Display Units Records
==========================
Load and display all of the records of type "units".

In [ ]:
print('Loading units records.')

# Extract the units records from the database
unit_recs = factory.createRecordDAO().get_all_of_type("units")

# Prepare to save off the units for later use
units_dict = {}

# Build a list of table entries, one unit per row
rows = [['Measurement', 'Units', 'Description']]

for unit in unit_recs:
    id_ = unit['id']
    data = unit['data']
    value = data['measure']['value']
    rows.append([id_, value, data['description']['value']])
    
    units_dict[id_] = value

# Display the rows in an HTML table
display_table(rows)

Load and Display Experiments Records
================================
Load and display all of the records of type "exp", which correspond to flights used to collect radiation measurements.

In [ ]:
print('Loading experiments records.')

# Extract experiment record(s) from the database
experiment_recs = factory.createRecordDAO().get_all_of_type("exp")

# Prepare to save off experiment ids for relationship queries
exp_ids = []

# Build a list of table entries, one experiment per table row
rows = [['Experiment Id', 'Data Source']]

for experiment in experiment_recs:
    id_ = experiment['id']
    exp_ids.append(id_)

    rows.append([id_, experiment['files'][0]['uri']])

# Display the rows in an HTML table
display_table(rows)

print('Ready to run the next cell.')

Load and Display Observation Records
===============================
Load and display all of the records of type "obs", which correspond to radiation measurements taken during one of the flights.

In [ ]:
print('Loading observation records.  This may take a while.')

# Extract the observation records from the database
observation_recs = factory.createRecordDAO().get_all_of_type("obs")

# Build a list of table entries, one observation per table row
rows = [
    ['Observation Id', 'Date', 'Time', 'Latitude', 'Longitude',
     'GPS Altitude','Sea Level Altitude', 'Live Time', 'Gross Counts']
]
scalars_of_interest = ['date', 'time', 'latitude', 'longitude', 'alt_hae',
              'agl', 'live', 'gcnorm']

# .. Add units to the header
for index, scalar in enumerate(scalars_of_interest):
    if scalar in units_dict.keys():
        rows[0][index+1] = '%s (%s)' % (rows[0][index+1], units_dict[scalar])

# .. Add observations to the output
for observation in observation_recs:
    values = [observation['id']]
    values.extend(['' for _ in scalars_of_interest]) # allow for missing values
    for index, scalar in enumerate(scalars_of_interest):
        values[index+1] = observation['data'][scalar]['value']
    rows.append(values)

# Display the rows in an HTML table
display_table(rows)

Load and Display Relationships
=========================
Load and display all of the relationships, which consist solely of relationships between experiments (i.e., flights) and observations (i.e., measurements).

In [ ]:
print('Loading relationships.  This may take a while.')

# Extract relationship records from the database, for each experiment
relationship_recs = factory.createRelationshipDAO()

# Build a list of table entries, one quality control entry per row
rows = [['Experiment Id', 'Relationship', 'Observation Id']]

for eid in exp_ids:
    relationships = relationship_recs.get(subject_id=eid)
    for relation in relationships:
        rows.append([eid, relation.predicate, relation.object_id])

# Display the rows in an HTML table
display_table(rows)